In [1]:
'''Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.callbacks import TensorBoard


import numpy as np
import random
import sys
import os


from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import LambdaCallback


from my_to_midi import *

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_integer('batch_size', 1024, 'LSTM Layer Units Number')
tf.app.flags.DEFINE_integer('epochs', 5, 'Total epochs')
tf.app.flags.DEFINE_integer('maxlen', 48, 'Max length of a sentence')
tf.app.flags.DEFINE_integer('step', 8, 'Step length when building dataset')
tf.app.flags.DEFINE_integer('embedding_length', 1, 'Embedding length')
tf.app.flags.DEFINE_string('dataset_name', 'Bach_train', 'Dataset name will be the prefix of exp_name')
tf.app.flags.DEFINE_string('dataset_path', 'datasets/Bach_train.txt', 'Dataset path')

In [2]:
batch_size = FLAGS.batch_size
epochs = FLAGS.epochs
units = FLAGS.units

maxlen = FLAGS.maxlen
step = FLAGS.step
embedding_length = FLAGS.embedding_length
dataset_name = FLAGS.dataset_name
dataset_path = FLAGS.dataset_path

exp_name = "%s_batchS%d_epochs%d_units%d_maxL%d_step%d_embeddingL%d" % (dataset_name, 
            batch_size, epochs, units, maxlen, step, embedding_length)

corpus length: 331708


In [ ]:
with open(dataset_text_path,"r") as f:
    text = f.read().lower()
print('corpus length:', len(text))

text = text[0:1000]

In [3]:
chars = "._0123456789abcdefghijklmnopqrstuvwxyz"
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
# wikifonia: 238w chars

total chars: 38


In [4]:
def text_to_events(str):
    ret = []
    for i in str:
        ret.append(char_indices[i])
    return ret

In [6]:
log_dir = os.path.join("logdir/", exp_name)
TB_log_dir = os.path.join('TB_logdir/', exp_name)
console_log_dir = os.path.join(log_dir, "console")
model_log_dir = os.path.join(log_dir, "model")
text_log_dir = os.path.join(log_dir, "text")
midi_log_dir = os.path.join(log_dir, "midi")


def make_log_dirs(dirs):
    for dir in dirs:
        if not os.path.exists(dir):
            os.makedirs(dir)

dirs = [log_dir, TB_log_dir, console_log_dir, model_log_dir, text_log_dir, midi_log_dir]
make_log_dirs(dirs)

max_acc_log_path = os.path.join("logdir/", "max_acc_log.txt")

In [7]:
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 119
Vectorization...


In [8]:
def lr_schedule(epoch):
    #Learning Rate Schedule
    lr = 1e-1
    if epoch >= epochs * 0.9:
        lr *= 0.5e-3
    elif epoch >= epochs * 0.8:
        lr *= 1e-3
    elif epoch >= epochs * 0.6:
        lr *= 1e-2
    elif epoch >= epochs * 0.4:
        lr *= 1e-1
    print('Learning rate: ', lr)

    lr = 1e-3
    return lr

In [9]:
def print_fn(str):
    print(str)
    console_log_file = os.path.join(console_log_dir, 'console_output.txt')
    with open(console_log_file, 'a+') as f:
        print(str, file=f)

In [10]:
# build the model: a single LSTM
print_fn('Build model...')

model = Sequential()
model.add(LSTM(units, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=lr_schedule(0))
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary(print_fn = print_fn)

Build model...
Learning rate:  0.1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               85504     
_________________________________________________________________
dense_1 (Dense)              (None, 38)                4902      
_________________________________________________________________
activation_1 (Activation)    (None, 38)                0         
Total params: 90,406
Trainable params: 90,406
Non-trainable params: 0
_________________________________________________________________


In [11]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [12]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    if epoch % (epochs // 5) != 0:
        return

    print_fn("")
    print_fn('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.4, 0.6, 0.8, 1.0, 1.2]:
        print_fn('----- diversity: %.1f' % diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print_fn('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

        log_name = "epoch%d_diversity%02d" % (epoch, int(diversity*10))
        text_log_path = os.path.join(text_log_dir, log_name+".txt")
        with open(text_log_path,"w") as text_log_file:
            text_log_file.write(generated+"\n")
            text_log_file.close()
        
        print_fn("Write %s.txt to %s"%(log_name, text_log_dir))
        
        events = text_to_events(generated)
        events_to_midi('basic_rnn', events, midi_log_dir, log_name)
        
        print_fn("Write %s.midi to %s" % (log_name, midi_log_dir) )
        
        model_name = "epoch%d.h5" % epoch
        model_path = os.path.join(model_log_dir, model_name)
        model.save(model_path)
        print_fn("Save model %s.h5 to %s" % (model_name, model_log_dir) )
        

In [14]:
lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)
# 参照下面代码加一下TensorBoard
tb_callbacks = TensorBoard(log_dir = TB_log_dir)
history_callback = model.fit(x, y,
          verbose = 0,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[tb_callbacks, lr_scheduler])

acc_history = history_callback.history["acc"]
max_acc = np.max(acc_history)
print_fn('Experiment %s max accuracy:%f' % (exp_name, max_acc) )
max_acc_log_line = "%s\t%d\t%d\t%d\t%d\t%d\t%f" % (exp_name, 
             epochs, units, maxlen, step, embedding_length, max_acc)

print(max_acc_log_line, file=open(max_acc_log_path, 'a'))

Learning rate:  0.1

Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
Learning rate:  0.1

Epoch 00002: LearningRateScheduler reducing learning rate to 0.001.
Learning rate:  0.010000000000000002

Epoch 00003: LearningRateScheduler reducing learning rate to 0.001.
Learning rate:  0.001

Epoch 00004: LearningRateScheduler reducing learning rate to 0.001.
Learning rate:  0.0001

Epoch 00005: LearningRateScheduler reducing learning rate to 0.001.
Experiment Bach_batchS1024_epochs5_units128_maxL48_step8_embeddingL1 max accuracy:0.142857
